In [2]:
import sys, os
sys.path.append('/home/danilofrp/projeto_final/neural-stocks/src')
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralstocks.dataacquisition import *
from neuralstocks.plots import *
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [4]:
dataPath = '/home/danilofrp/projeto_final/data'
assetType = 'stocks'
asset = 'PETR4'
frequency = 'diario'
filePath = dataPath + '/' + assetType + '/' + asset + '/' + frequency + '/' + asset + '.CSV'

decomposeModel = 'additive'

saveDir = '/home/danilofrp/projeto_final/results/preprocessing/misc'
if not os.path.exists(saveDir):
    os.makedirs(saveDir)
saveFormat = 'png'

plt.rcParams['font.weight'] = 'bold'
plt.rcParams['figure.titlesize'] = 18
plt.rcParams['figure.titleweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13

In [5]:
init_time = time.time()
PETR4 = acquireData(filePath = filePath,
                    replicateForHolidays = True,
                    meanStdLen = 20,
                    returnCalcParams = [['Close'], ['Close', 'Open']],
                    EMAparams = [{'column': 'Close', 'lenght': 17}, 
                                 {'column': 'Close', 'lenght': 72}, 
                                 {'column': 'Close', 'lenght': 200}, 
                                 {'column': 'Volume', 'lenght': 21}],
                    MACDParams = [{'fast_lenght': 12, 'slow_lenght': 26, 'signal_lenght': 9}],
                    BBParams = [{'lenght': 20}],
                    OBVParams = [{'lenght': None}],
                    deTrendParams = {'column': 'Close', 'window': 6, 'model': decomposeModel, 
                                     'weightModel': 'window_acorr', 'weightModelWindow': 18},
                    colPrefix = None,
                    dropNan = True)

columnsToKeep = ['Holiday', 'Close_rollStd20', 'Close_EMA17_logdiff', 'Close_EMA72_logdiff',
                 'Close_EMA200_logdiff', 'Volume_EMA21_logdiff', 'MACD_12_26_9', 'MACDsignal_12_26_9', 'Bollinger%b_20',
                 'OBV', 'Close_returns', 'Close/Open_returns', 'Close_resid']
PETR4 = PETR4[columnsToKeep]

end_time = time.time()
print 'Time to acquire and process data: '+str(end_time-init_time)+' seconds'

Time to acquire and process data: 15.2587380409 seconds


In [ ]:
plotSeries(PETR4[['Close_EMA17_logdiff', 'Close_EMA72_logdiff', 'Close_EMA200_logdiff']], 
           title = 'Relative position of {} Close to 17-, 72- and 200-period EMA'.format(asset), ylabel = 'Price', 
           initialPlotDate = '2015', finalPlotDate = '2016')

In [6]:
PETR4_scalled = None
scaler = MinMaxScaler(feature_range = (-1,1))
scaler.fit(PETR4)
PETR4_scalled = pd.DataFrame(scaler.transform(PETR4), columns = PETR4.columns.values, index = PETR4.index)

#scatterHist(PETR4['Close_EMA17_logdiff'], PETR4_scalled['Close_EMA17_logdiff'], nBins = 100)

In [40]:
#from keras.models import Sequential
#from keras.layers import Dense

#model = Sequential()
#model.add(Dense())

xTrain, yTrain, xTest, yTest = prepData2(df = PETR4, 
                                        columnsToUse = columnsToKeep, columnToPredict = 'Close_resid', 
                                        nDelays = 10, testSetSize = len(PETR4['2017']))

print PETR4['Close_resid'].head(11)

print '{}: {}'.format(xTrain[0], yTrain[0])


Date
2000-12-04    0.049945
2000-12-05    0.069956
2000-12-06   -0.199005
2000-12-07    0.010717
2000-12-08    0.045235
2000-12-11    0.051263
2000-12-12    0.048916
2000-12-13    0.185498
2000-12-14    0.062907
2000-12-15   -0.028874
2000-12-18   -0.099069
Name: Close_resid, dtype: float64
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11158664323013918, 0.11316034269078852, 0.13619258809727336, 0.15099233162822648, 0.16254230218623711, 0.17149113833174542, 0.17781939388634566, 0.17518336258423028, 0.16766115578122703, 0.15860908582192046, -0.047036163670576107, -0.030246761696000348, -0.090146618759716926, -0.068109072680145391, -0.060766467930280406, -0.057274732770613394, -0.0541605838342816, -0.0060951751191548253, 0.0033389656351548815, 5.8346711833017268e-05, -0.072295896850299482, -0.057640993725745097, -0.1249427068376938, -0.10820601407045768, -0.10538221569585324, -0.10598573813925508, -0.10658620981513178, -0.057696753582009534, -0.046559603007906389, -0.0484960754005